# Initialize

In [ ]:
import findspark
findspark.init("/usr/lib/spark-3.2/")

In [ ]:
import math
import time

In [ ]:
qid='s072'
appid='s072_application_1710358003975_2151'
vanilla_appid='s072_application_1710358003975_2152'

In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = "./jupyter_env/bin/python"

In [ ]:
executors_per_node = 5
nodes=2
cores_per_executor=4
task_per_core=4


cache_size=20
total_size=100000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = (SparkConf()
    .set("spark.default.parallelism", "{:d}".format(nodes*executors_per_node))
    .set('spark.executor.instances', '{:d}'.format(nodes*executors_per_node))
    .set('spark.sql.files.maxPartitionBytes', '256m')
    .set('spark.app.name', f'{qid}_process')
    .set('spark.rdd.compress', 'False')
    .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')    
    .set('spark.executor.cores','{:d}'.format(cores_per_executor))
    .set('spark.sql.adaptive.coalescePartitions.initialPartitionNum','{:d}'.format(nodes*executors_per_node*cores_per_executor*task_per_core))
    .set('spark.sql.adaptive.enabled',True)
    .set('spark.sql.adaptive.advisoryPartitionSizeInBytes', '256m')
    .set('spark.executor.memory', '{:d}m'.format(int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.task.cpus','1')
    .set('spark.driver.memory','128g')
    .set('spark.sql.inMemoryColumnarStorage.compressed','False')
    .set('spark.sql.inMemoryColumnarStorage.batchSize','100000')
#    .set('spark.memory.storageFraction','0.8')
#    .set('spark.memory.fraction','0.7')
        
    .set('spark.sql.execution.arrow.fallback.enabled','True')
    .set('spark.sql.execution.arrow.enabled','True')
    .set('spark.sql.execution.arrow.maxRecordsPerBatch','100000')
    .set("spark.sql.repl.eagerEval.enabled", True)
        
    .set('spark.memory.offHeap.enabled','True')
    .set('spark.memory.offHeap.size','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.executor.memoryOverhead','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))+3000))
    .set('spark.sql.join.preferSortMergeJoin','False')
#    .set('spark.executor.extraJavaOptions',
#          '-XX:+UseG1GC -XX:+PrintFlagsFinal -XX:+PrintReferenceGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps -XX:InitiatingHeapOccupancyPercent=35')
    .set('spark.executor.extraJavaOptions',
          '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps')
    .set('spark.dynamicAllocation.enabled', 'False')
    .set('spark.shuffle.service.enabled', 'False')
    .set('spark.dynamicAllocation.shuffleTracking.enabled', 'Falze')
    .set("spark.sql.legacy.timeParserPolicy","LEGACY")
    .set("spark.kryoserializer.buffer.max","1024m")
    .set('spark.sql.autoBroadcastJoinThreshold', '-1')

        
#    .set("spark.sql.session.timeZone", "PST")
     .set("spark.sql.session.timeZone", "Etc/GMT-0")
#     .set("spark.driver.extraJavaOptions","-Duser.timezone=UTC+8")
    .set('spark.yarn.dist.archives', "/home/binweiyang/jupyter_env.tar.gz#jupyter_env")
       )

sc = SparkContext(conf=conf,master='yarn')
sc.setLogLevel("ERROR")
spark = SQLContext(sc)
time.sleep(10)


In [ ]:
%run /home/binweiyang/ipython/spark_log_pinterest.ipynb

# analysis

In [ ]:
app=Application_Run(appid,basedir="profile")
appals=app.analysis['app']['als']

In [ ]:
appals.get_basic_state()

In [ ]:
summary=app.get_summary(disk_prefix=["'nvme1n1'","'nvme2n1'"],nic_prefix=["'eth0'"])
display(summary.style)

In [ ]:
app.generate_trace_view(disk_prefix=["'nvme1n1'","'nvme2n1'"],nic_prefix=["'eth0'"])

In [ ]:
bkd=appals.show_critical_path_time_breakdown()

In [ ]:
bkd.to_dict()

In [ ]:
qtime=appals.get_query_time()

In [ ]:
for qid,time in dict(qtime["runtime"]).items():
    if time>100:
        appals.get_query_plan(queryid=qid,show_simple_string=True)

In [ ]:
app_base=Application_Run(vanilla_appid,basedir="profile")
for qid,time in dict(qtime["runtime"]).items():
    if time>100:
        app.show_queryplan_diff(qid,rapp=app_base)


In [ ]:
t=appals.get_hottest_stages()


In [ ]:
app_base=Application_Run(vanilla_appid,basedir="profile")
t=app_base.analysis['app']['als'].get_hottest_stages()

In [20]:
sc.stop()